<a href="https://colab.research.google.com/github/LES4975/Album_Art_Generator_Project/blob/main/Album_Art_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

첫 번째 셀은 인터페이스만 표시하는 데모 코드입니다. 시연할 때 실행하지 않아도 됩니다.

In [1]:
# gradio_ui_demo.py
# Gradio 인터페이스 미리보기 전용 (실제 기능 없이 UI만)

import gradio as gr
from pathlib import Path
import time
from PIL import Image
import numpy as np

def demo_file_upload(audio_file):
    """파일 업로드 시 제목 감지 (데모)"""
    if audio_file is None:
        return ""

    music_title = Path(audio_file).stem
    return music_title

def demo_generate_album_art(audio_file):
    """앨범 아트 생성 (데모 - 실제로는 더미 데이터 반환)"""

    if audio_file is None:
        return None, "❌ 음악 파일을 업로드해주세요", "", "", "", gr.update(visible=False)

    # 파일명에서 제목 추출
    music_title = Path(audio_file).stem

    # 진행 상태 시뮬레이션
    yield None, "🔄 음악 분석 중...", "", "", music_title, gr.update(visible=False)
    time.sleep(1)

    yield None, "🔄 프롬프트 생성 중...", "", "", music_title, gr.update(visible=False)
    time.sleep(1)

    yield None, "🔄 앨범 아트 생성 중...", "", "", music_title, gr.update(visible=False)
    time.sleep(2)

    # 더미 이미지 생성 (데모용)
    dummy_image = Image.fromarray(
        np.random.randint(0, 255, (512, 512, 3), dtype=np.uint8)
    )

    # 더미 분석 결과
    demo_analysis = f"""⏱️ 길이: 3.2초

🎼 상위 장르:
  1. electronic: 0.856
  2. ambient: 0.743
  3. experimental: 0.621

🎭 주요 분위기:
  • calm: 0.789
  • meditative: 0.654
  • atmospheric: 0.587

🎨 주요 테마:
  • space: 0.723
  • nature: 0.612

📊 상위 분위기 (전체):
  1. calm: 0.789
  2. space: 0.723
  3. meditative: 0.654
  4. nature: 0.612
  5. atmospheric: 0.587"""

    # 더미 프롬프트
    demo_prompt = f"Album cover for '{music_title}', electronic genre, calm mood, space theme, artistic album cover design, professional music artwork, high quality, detailed illustration"

    # 더미 다운로드 파일 (실제로는 생성된 이미지)
    download_update = gr.update(visible=True, value=None)  # 데모에서는 실제 파일 없음

    return dummy_image, "✅ 앨범 아트 생성 완료! (데모)", demo_analysis, demo_prompt, music_title, download_update

def create_demo_interface():
    """Gradio 인터페이스 데모 생성"""

    with gr.Blocks(title="🎵 Music-Based Album Art Generator (Demo)", theme=gr.themes.Soft()) as demo:

        # 헤더
        gr.Markdown("""
        # 🎵 Music-Based Album Art Generator (UI 데모)

        **⚠️ 이것은 UI 미리보기용 데모입니다. 실제 음악 분석이나 이미지 생성은 수행되지 않습니다.**

        음악을 업로드하면 자동으로 분석하여 어울리는 앨범 아트를 생성합니다!

        **사용 방법:**
        1. 음악 파일을 업로드하세요 (MP3, WAV 등)
        2. 파일명이 곡 제목으로 자동 감지됩니다
        3. "앨범 아트 생성" 버튼을 클릭하세요
        4. 결과를 확인하고 다운로드하세요!
        """, elem_classes="header")

        with gr.Row():
            # 입력 섹션
            with gr.Column(scale=1):
                gr.Markdown("### 📤 입력")

                audio_input = gr.File(
                    label="🎵 음악 파일 업로드",
                    file_types=[".mp3", ".wav", ".m4a", ".flac"],
                    type="filepath"
                )

                title_display = gr.Textbox(
                    label="🎼 감지된 곡 제목",
                    placeholder="파일 업로드 시 자동 감지됩니다",
                    interactive=False,
                    value=""
                )

                generate_btn = gr.Button(
                    "🎨 앨범 아트 생성 (데모)",
                    variant="primary",
                    size="lg"
                )

            # 출력 섹션
            with gr.Column(scale=2):
                gr.Markdown("### 🖼️ 결과")

                status_output = gr.Textbox(
                    label="📋 상태",
                    value="음악 파일을 업로드하고 생성 버튼을 눌러주세요",
                    interactive=False
                )

                image_output = gr.Image(
                    label="🎨 생성된 앨범 아트 (데모 - 랜덤 이미지)",
                    type="pil",
                    height=400
                )

                download_btn = gr.DownloadButton(
                    label="💾 앨범 아트 다운로드 (데모용 - 동작하지 않음)",
                    visible=False
                )

        # 상세 정보 섹션 (접을 수 있음)
        with gr.Accordion("🔍 상세 분석 결과 (데모)", open=False):
            analysis_output = gr.Textbox(
                label="📊 음악 분석 결과 (데모 데이터)",
                lines=10,
                interactive=False
            )

            prompt_output = gr.Textbox(
                label="📝 생성된 프롬프트 (데모)",
                lines=3,
                interactive=False
            )

        # 이벤트 연결
        # 파일 업로드 시 제목 자동 감지
        audio_input.change(
            fn=demo_file_upload,
            inputs=[audio_input],
            outputs=[title_display]
        )

        # 버튼 이벤트 연결
        generate_btn.click(
            fn=demo_generate_album_art,
            inputs=[audio_input],
            outputs=[image_output, status_output, analysis_output, prompt_output, title_display, download_btn]
        )

        # 예시 섹션
        with gr.Row():
            gr.Markdown("""
            ### 💡 팁
            - **지원 형식**: MP3, WAV, M4A, FLAC
            - **권장 길이**: 30초 이상 (더 정확한 분석)
            - **파일명 자동 감지**: 업로드된 파일명이 곡 제목으로 사용됩니다
            - **생성 시간**: GPU 사용시 약 10-30초, CPU 사용시 1-3분

            ### 🎯 프로젝트 정보
            - **음악 분석**: Essentia + 사전학습된 모델
            - **이미지 생성**: Stable Diffusion v1.5
            - **개발 환경**: Google Colab

            ### ⚠️ 데모 안내
            이것은 UI 미리보기용 데모입니다:
            - 실제 음악 분석은 수행되지 않습니다
            - 랜덤 이미지가 표시됩니다
            - 더미 분석 결과가 표시됩니다
            - 다운로드는 동작하지 않습니다
            """)

    return demo

# 실행 함수
def launch_demo():
    """데모 실행"""
    print("🎵 Gradio UI 데모 시작")
    print("=" * 50)
    print("⚠️ 이것은 UI 미리보기용 데모입니다")
    print("실제 기능은 동작하지 않으며, 더미 데이터가 표시됩니다")
    print("=" * 50)

    demo = create_demo_interface()

    return demo.launch(
        share=True,  # 공유 링크 생성
        debug=True,
        show_error=True
    )

# 직접 실행시
if __name__ == "__main__":
    launch_demo()

🎵 Gradio UI 데모 시작
⚠️ 이것은 UI 미리보기용 데모입니다
실제 기능은 동작하지 않으며, 더미 데이터가 표시됩니다
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://93d1f8d7f907aecfb7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://93d1f8d7f907aecfb7.gradio.live


In [ ]:
# main.py
# 메인 실행 스크립트 - 모든 초기화 작업 포함

"""
Album Art Generator - 메인 실행 파일

사용법:
1. Google Colab에서 이 파일을 실행
2. 모든 초기화가 자동으로 진행됩니다
3. Gradio 앱이 자동으로 시작됩니다
"""

import os
import sys
from pathlib import Path
import subprocess

def install_libraries():
    """필요한 라이브러리 설치"""
    print("📦 라이브러리 설치 중...")

    # 라이브러리 목록
    libraries = [
        "gradio",
        "diffusers",
        "transformers",
        "accelerate",
        '"numpy==1.26.4"',  # numpy 버전 고정
        "essentia-tensorflow --no-deps",  # Essentia 설치
        "pyyaml",
        "six"
    ]

    for lib in libraries:
        try:
            print(f"  설치 중: {lib}")
            subprocess.run(f"pip install {lib} --force-reinstall", shell=True, check=True, capture_output=True)
        except subprocess.CalledProcessError as e:
            print(f"  ⚠️ {lib} 설치 실패: {e}")

    print("✅ 라이브러리 설치 완료")

def setup_environment():
    """환경 설정"""
    print("🔧 환경 설정 중...")

    # 1. 라이브러리 설치
    install_libraries()

    # 2. Google Drive 마운트
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive 마운트 완료")
    except ImportError:
        print("⚠️ Google Colab 환경이 아닙니다")
        return False
    except Exception as e:
        print(f"❌ Google Drive 마운트 실패: {e}")
        return False

    print("✅ 환경 설정 완료")
    return True


In [ ]:
def setup_model_files():
    """모델 파일 설정 및 심볼링크 생성"""
    print("🔗 모델 파일 설정 중...")

    # 기본 경로
    drive_path = "/content/drive/MyDrive/album_art_generator_project"

    # 모델 파일 경로들
    model_files = {
        "discogs_model": f"{drive_path}/discogs-effnet-bs64-1.pb",
        "mood_model": f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.pb",
        "discogs_json": f"{drive_path}/discogs-effnet-bs64-1.json",
        "mood_json": f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.json"
    }

    # 파일 존재 확인
    missing_files = []
    for name, file_path in model_files.items():
        if Path(file_path).exists():
            print(f"✅ {name}: 존재함")
        else:
            print(f"❌ {name}: 없음")
            missing_files.append(file_path)

    if missing_files:
        print(f"\n⚠️ 누락된 모델 파일 {len(missing_files)}개:")
        for file_path in missing_files:
            print(f"   - {file_path}")
        return False

    # 작업 디렉토리에 심볼링크 생성
    try:
        symlinks = [
            (model_files["discogs_model"], "discogs-effnet-bs64-1.pb"),
            (model_files["mood_model"], "mtg_jamendo_moodtheme-discogs-effnet-1.pb"),
            (model_files["discogs_json"], "discogs-effnet-bs64-1.json"),
            (model_files["mood_json"], "mtg_jamendo_moodtheme-discogs-effnet-1.json")
        ]

        for source, target in symlinks:
            if not Path(target).exists():
                os.symlink(source, target)
                print(f"🔗 심볼링크 생성: {target}")
            else:
                print(f"✅ 심볼링크 존재: {target}")

        print("✅ 모델 파일 설정 완료")
        return True

    except Exception as e:
        print(f"❌ 심볼링크 생성 실패: {e}")
        return False

In [ ]:
def check_files():
    """필수 파일 존재 확인"""
    print("📁 파일 확인 중...")

    # 기본 경로
    drive_path = "/content/drive/MyDrive/album_art_generator_project"

    # 필수 파일들
    required_files = [
        f"{drive_path}/discogs-effnet-bs64-1.pb",
        f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.pb",
        f"{drive_path}/discogs-effnet-bs64-1.json",
        f"{drive_path}/mtg_jamendo_moodtheme-discogs-effnet-1.json",
        f"{drive_path}/music_classification_colab.py"
    ]

    missing_files = []
    for file_path in required_files:
        if Path(file_path).exists():
            print(f"✅ {Path(file_path).name}")
        else:
            print(f"❌ {Path(file_path).name}")
            missing_files.append(file_path)

    if missing_files:
        print(f"\n⚠️ 누락된 파일 {len(missing_files)}개:")
        for file_path in missing_files:
            print(f"   - {file_path}")
        print("\n해결 방법:")
        print("1. Google Drive에 필요한 파일들을 업로드하세요")
        print("2. 파일 경로가 올바른지 확인하세요")
        return False

    print("✅ 모든 필수 파일 확인 완료")
    return True



In [ ]:
def initialize_music_classifier():
    """음악 분류기 초기화"""
    print("🎵 음악 분류기 초기화 중...")

    try:
        # 경로 추가
        sys.path.append('/content/drive/MyDrive/album_art_generator_project')

        # Essentia import 테스트
        try:
            import essentia
            print(f"✅ Essentia 버전: {essentia.__version__}")
        except Exception as e:
            print(f"❌ Essentia 로드 실패: {e}")
            print("Runtime을 재시작해야 할 수 있습니다")
            return None

        # 음악 분류기 생성
        from music_classification_colab import ColabMusicClassifier
        classifier = ColabMusicClassifier()

        print("✅ 음악 분류기 초기화 완료")
        return classifier

    except Exception as e:
        print(f"❌ 음악 분류기 초기화 실패: {e}")
        return None

def test_system():
    """시스템 테스트"""
    print("🧪 시스템 테스트 중...")

    try:
        # numpy 버전 확인
        import numpy as np
        print(f"📊 numpy 버전: {np.__version__}")

        # Essentia 테스트
        import essentia
        print(f"🎵 Essentia 버전: {essentia.__version__}")

        # torch 확인 (GPU)
        import torch
        print(f"🔥 PyTorch GPU 사용 가능: {torch.cuda.is_available()}")

        print("✅ 시스템 테스트 완료")
        return True

    except Exception as e:
        print(f"❌ 시스템 테스트 실패: {e}")
        return False


In [ ]:
def main():
    """메인 실행 함수"""
    print("🎵 Album Art Generator 시작")
    print("=" * 50)

    # 1. 환경 설정
    if not setup_environment():
        print("❌ 환경 설정 실패로 종료합니다")
        return

    # 2. 파일 확인
    if not check_files():
        print("❌ 필수 파일이 누락되어 실행을 중단합니다")
        return

    # 3. 모델 파일 설정
    if not setup_model_files():
        print("❌ 모델 파일 설정 실패로 종료합니다")
        return

    # 4. 시스템 테스트
    if not test_system():
        print("❌ 시스템 테스트 실패. Runtime 재시작 후 다시 시도하세요")
        return

    # 5. 음악 분류기 초기화
    classifier = initialize_music_classifier()
    if classifier is None:
        print("❌ 음악 분류기 초기화 실패로 종료합니다")
        return

    # 6. 앱 실행
    try:
        print("🚀 Gradio 앱 시작...")

        # 모듈 임포트
        from gradio_app import create_and_launch_app

        # 앱 실행 (초기화된 분류기 전달)
        print("🎯 앱 시작...")
        create_and_launch_app(
            music_classifier_path="/content/drive/MyDrive/album_art_generator_project",
            classifier_instance=classifier  # 초기화된 분류기 전달
        )

    except ImportError as e:
        print(f"❌ 모듈 임포트 실패: {e}")
        print("album_art_core.py와 gradio_app.py 파일이 현재 디렉토리에 있는지 확인하세요")
    except Exception as e:
        print(f"❌ 앱 실행 실패: {e}")

def restart_runtime_warning():
    """Runtime 재시작 안내"""
    print("\n" + "="*60)
    print("⚠️  RUNTIME 재시작이 필요할 수 있습니다")
    print("="*60)
    print("만약 Essentia 관련 오류가 발생한다면:")
    print("1. Runtime > Restart runtime 클릭")
    print("2. 이 스크립트를 다시 실행하세요")
    print("="*60)


In [ ]:
if __name__ == "__main__":
    restart_runtime_warning()

    # 메인 실행
    main()